In [5]:
import cv2
import os

video_path = "C:/Users/Admin/Documents/Uni/sem5/CV/tf2.mp4" 
output_dir = "C:/Users/Admin/Documents/Uni/sem5/CV/dataset/color"  
os.makedirs(output_dir, exist_ok=True)

# Load video
cap = cv2.VideoCapture(video_path)
frame_rate = 2
frame_count = 0
saved_count = 0

while cap.isOpen():
    ret, frame = cap.read()
    if not ret:
        break
    
    if frame_count % frame_rate == 0:
        frame_filename = os.path.join(output_dir, f"frame_{saved_count:04d}.jpg")
        cv2.imwrite(frame_filename, frame)
        saved_count += 1
        print(f"Saved: {frame_filename}")
    
    frame_count += 1

cap.release()

cap = cv2.VideoCapture(video_path)
frame_rate = 1
frame_count = 0
saved_count = 9249

while frame_count < 751:
    ret, frame = cap.read()
    if not ret:
        break
    
    if frame_count % frame_rate == 0:
        frame_filename = os.path.join(output_dir, f"frame_{saved_count:04d}.jpg")
        cv2.imwrite(frame_filename, frame)
        saved_count += 1
        print(f"Saved: {frame_filename}")
    
    frame_count += 1

cap.release()
print(f"Total frames saved: {saved_count}")


Saved: C:/Users/Admin/Documents/Uni/sem5/CV/dataset/color\frame_9249.jpg
Saved: C:/Users/Admin/Documents/Uni/sem5/CV/dataset/color\frame_9250.jpg
Saved: C:/Users/Admin/Documents/Uni/sem5/CV/dataset/color\frame_9251.jpg
Saved: C:/Users/Admin/Documents/Uni/sem5/CV/dataset/color\frame_9252.jpg
Saved: C:/Users/Admin/Documents/Uni/sem5/CV/dataset/color\frame_9253.jpg
Saved: C:/Users/Admin/Documents/Uni/sem5/CV/dataset/color\frame_9254.jpg
Saved: C:/Users/Admin/Documents/Uni/sem5/CV/dataset/color\frame_9255.jpg
Saved: C:/Users/Admin/Documents/Uni/sem5/CV/dataset/color\frame_9256.jpg
Saved: C:/Users/Admin/Documents/Uni/sem5/CV/dataset/color\frame_9257.jpg
Saved: C:/Users/Admin/Documents/Uni/sem5/CV/dataset/color\frame_9258.jpg
Saved: C:/Users/Admin/Documents/Uni/sem5/CV/dataset/color\frame_9259.jpg
Saved: C:/Users/Admin/Documents/Uni/sem5/CV/dataset/color\frame_9260.jpg
Saved: C:/Users/Admin/Documents/Uni/sem5/CV/dataset/color\frame_9261.jpg
Saved: C:/Users/Admin/Documents/Uni/sem5/CV/dataset

In [6]:
# input_dir = "C:/Users/Admin/Documents/Uni/sem5/CV/dataset/grayscale"
# os.makedirs(input_dir, exist_ok=True)

# for file in os.listdir(output_dir):
#     img_path = os.path.join(output_dir, file)
#     img = cv2.imread(img_path)
    
#     gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
#     gray_img = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2BGR)
    
#     cv2.imwrite(os.path.join(input_dir, file), gray_img)

# print("Grayscale dataset created.")

Grayscale dataset created.


In [7]:
# import shutil
# from sklearn.model_selection import train_test_split

# train_dir = "C:/Users/Admin/Documents/Uni/sem5/CV/dataset/train"
# test_dir = "C:/Users/Admin/Documents/Uni/sem5/CV/dataset/test"

# for d in [train_dir, test_dir]:
#     os.makedirs(os.path.join(d, "color"), exist_ok=True)
#     os.makedirs(os.path.join(d, "grayscale"), exist_ok=True)

# all_files = sorted(os.listdir(output_dir))  
# train_files, test_files = train_test_split(all_files, test_size=0.2, random_state=42)

# for file in train_files:
#     shutil.copy(os.path.join(output_dir, file), os.path.join(train_dir, "color", file))
#     shutil.copy(os.path.join(input_dir, file), os.path.join(train_dir, "grayscale", file))

# for file in test_files:
#     shutil.copy(os.path.join(output_dir, file), os.path.join(test_dir, "color", file))
#     shutil.copy(os.path.join(input_dir, file), os.path.join(test_dir, "grayscale", file))

# print("Dataset split into training and testing sets.")


Dataset split into training and testing sets.


In [8]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np

class ColorizationNet(nn.Module):
    def __init__(self):
        super(ColorizationNet, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()  
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

model = ColorizationNet()
print(model)


ColorizationNet(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): Sigmoid()
  )
)


In [9]:
class ColorizationDataset(Dataset):
    def __init__(self, grayscale_dir, color_dir, transform=None):
        self.grayscale_files = sorted(os.listdir(grayscale_dir))
        self.color_files = sorted(os.listdir(color_dir))
        self.grayscale_dir = grayscale_dir
        self.color_dir = color_dir
        self.transform = transform

    def __len__(self):
        return len(self.grayscale_files)

    def __getitem__(self, idx):
        gray_img = Image.open(os.path.join(self.grayscale_dir, self.grayscale_files[idx])).convert("RGB")
        color_img = Image.open(os.path.join(self.color_dir, self.color_files[idx])).convert("RGB")

        if self.transform:
            gray_img = self.transform(gray_img)
            color_img = self.transform(color_img)

        return gray_img, color_img

transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
])

train_dataset = ColorizationDataset("coloring/data/train/grayscale", "coloring/data/train/color", transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    for gray, color in train_loader:
        optimizer.zero_grad()
        output = model(gray)
        loss = criterion(output, color)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")

print("Training complete.")


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

test_dataset = ColorizationDataset("dataset/test/grayscale", "dataset/test/color", transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

gray_img, color_img = next(iter(test_loader))
output = model(gray_img).detach().numpy()

gray_img = gray_img.numpy().squeeze().transpose(1, 2, 0)
color_img = color_img.numpy().squeeze().transpose(1, 2, 0)
output_img = output.squeeze().transpose(1, 2, 0)

fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(gray_img, cmap="gray")
ax[0].set_title("Input (Grayscale)")

ax[1].imshow(output_img)
ax[1].set_title("Model Output (Colorized)")

ax[2].imshow(color_img)
ax[2].set_title("Target (Original)")

plt.show()